# Setup

In [ ]:
import asyncio
import queue

In [ ]:
!pip install memory_profiler

In [ ]:
import memory_profiler
import time

def time_mem_decorator(func):                                                                                            
    def out(*args, **kwargs):                                                                                            
        m1 = memory_profiler.memory_usage()
        t1 = time.time()
        
        result = func(*args, **kwargs)
        
        t2 = time.time()
        m2 = memory_profiler.memory_usage()
        time_diff = t2 - t1
        mem_diff = m2[0] - m1[0]
        print(f"It took {time_diff} Secs and {mem_diff} Mb to execute this function.")
        return(result)
    return out 

# Wachten op taken

We definiëren hier een taak waar op gewacht moet worden, voor er resultaat is. Een praktisch voorbeeld hiervan is het wachten op een response van een server na het laten uitvoeren van een ingewikkelde berekening.

In [ ]:
def calculation(x):
    print(f"Running calculation with x = {x}.")
    time.sleep(x)
    print(f"Task with x = {x} is done.")

We voeren de taak uit voor een lijst van 5. Als we dit op de normale manier doen (synchronous), moeten we steeds wachten tot de vorige taak klaar is voor we de volgende kunnen aanroepen.

In [ ]:
@time_mem_decorator
def synchronous_work():
    task_queue = queue.Queue()
    
    for t in [2, 5, 10, 4, 6]:
        task_queue.put(t)
        
    while not task_queue.empty():
        x = task_queue.get()
        calculation(x)
    


In [ ]:
synchronous_work()

Als er meerdere taken tegelijk kunnen worden uitgevoerd, is het erg inefficiënt om te wachten op resultaat voor we beginnen met het uitvoeren van de volgende taak. Gelukkig kunnen we in Python gebruik maken van de async library!

Als we code asynchronous uitvoeren, kunnen we alvast verder gaan met de executie van de code terwijl we wachten op resultaat. Er wordt niet gewacht tot de volgende operatie klaar is. 

In [ ]:
async def async_calculation(x):
    print(f"Running calculation with x = {x}.")
    await asyncio.sleep(x)
    print(f"Task with x = {x} is done.")

We maken een worker functie aan die de referentie naar een queue als argument krijgt. Deze worker voert nog steeds stap voor stap instructies uit. Het grote verschil is dat de sleep functie niet het programma blokkeert. 

We optimaliseren de code door meerdere workers aan te maken. Het hoofd programma start beide workers, welke onafhankelijk van elkaar hun taken gaan uitvoeren. Terwijl een worker sleeped, kan een andere worker gewoon zijn eigen taak starten. 

In [ ]:
async def worker(task_queue):
    while not task_queue.empty():
        x = await task_queue.get()
        await async_calculation(x)
        task_queue.task_done()

In [ ]:
async def asynchronous_work():
    t1 = time.time()
    task_queue = asyncio.Queue()
    
    for t in [2, 5, 10, 4, 6]:
        await task_queue.put(t)
       
    await asyncio.gather(
        asyncio.create_task(worker(task_queue)),
        asyncio.create_task(worker(task_queue))
    )
    t2 = time.time()
    
    print(f"It took {t2 - t1} Secs to execute this function.")

In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
def main():
  loop = asyncio.get_event_loop()
  return asyncio.gather(loop.create_task(asynchronous_work()))

In [ ]:
main()

In [ ]:
event_loop = asyncio.get_event_loop()
asyncio.gather(event_loop.create_task(asynchronous_work()))
